### 什么是Embedding层？

在自然语言处理中，文本数据通常以词或字符的形式存在，而计算机无法直接处理这些离散的符号。因此，**Embedding层**的作用是将这些离散的词转换为连续的、高维的数值向量，这些向量通常能够捕捉词语之间的语义关系。

### Embedding层的基本原理：
1. **输入**: 一个整数序列（代表单词索引）。
2. **映射**: 通过一个矩阵，将输入索引映射到一个固定大小的向量（查找矩阵的一行）。
3. **输出**: 词向量，通常用于后续RNN层的输入。

### 让我们手工实现一个Embedding层：

#### 1. 依赖库

In [7]:
import numpy as np

#### 2. 初始化Embedding矩阵
假设我们有一个词汇表大小为 `vocab_size = 10`，每个词将被映射到一个维度为 `embedding_dim = 4` 的向量。

In [8]:
class SimpleEmbedding:
    def __init__(self, vocab_size, embedding_dim):
        # 随机初始化嵌入矩阵（大小：vocab_size x embedding_dim）
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.embeddings = np.random.randn(vocab_size, embedding_dim)
    
    def forward(self, input_indices):
        """
        input_indices: 一个包含词索引的数组，例如 [1, 3, 5]
        返回: 嵌入向量的数组
        """
        return self.embeddings[input_indices]
    
    def backward(self, input_indices, grad_output, learning_rate=0.01):
        """
        执行反向传播：更新嵌入矩阵的权重。
        
        input_indices: 输入的单词索引列表
        grad_output: 上游的梯度
        learning_rate: 学习率
        """
        for i, index in enumerate(input_indices):
            self.embeddings[index] -= learning_rate * grad_output[i]

#### 3. 测试Embedding层的前向传播

In [9]:
# 创建Embedding层
embedding_layer = SimpleEmbedding(vocab_size=10, embedding_dim=4)

# 假设输入是词索引序列 [1, 3, 5]
input_indices = np.array([1, 3, 5])
embedded_vectors = embedding_layer.forward(input_indices)

print("输入索引:", input_indices)
print("对应的嵌入向量:\n", embedded_vectors)

输入索引: [1 3 5]
对应的嵌入向量:
 [[ 1.75698167  0.53049996  0.75759184 -1.86272847]
 [-1.06185763  1.02098985  0.89978505 -0.07378516]
 [-1.12500159  0.83426525 -0.83575991 -0.60213772]]


#### 4. 反向传播（模拟训练过程）
假设从上层得到了损失函数的梯度：

In [10]:
grad_output = np.array([
    [0.1, 0.2, -0.1, 0.05],  # 对应第1个单词的梯度
    [0.01, -0.05, 0.2, -0.1],  # 对应第3个单词的梯度
    [-0.1, 0.05, 0.1, -0.2]   # 对应第5个单词的梯度
])

# 执行反向传播更新嵌入矩阵
embedding_layer.backward(input_indices, grad_output, learning_rate=0.01)

print("更新后的嵌入矩阵:\n", embedding_layer.embeddings)

更新后的嵌入矩阵:
 [[ 1.45538239  0.5496268  -0.74792845  0.68092713]
 [ 1.75598167  0.52849996  0.75859184 -1.86322847]
 [ 1.74324464  0.93800379  0.32398963  0.17839578]
 [-1.06195763  1.02148985  0.89778505 -0.07278516]
 [-0.00582238  0.0865723  -0.46563895 -0.50950293]
 [-1.12400159  0.83376525 -0.83675991 -0.60013772]
 [ 1.74711793  0.43752603 -0.44210632 -0.42132947]
 [ 0.72313972  0.62019241  0.30399549  0.95957268]
 [ 1.21556613 -0.35346932 -1.07034161 -0.36901774]
 [ 0.66738462  0.47894538  1.35162036 -0.89991466]]


### 现在来尝试处理一下文本

比如，把下面一首儿歌转换为词向量应该怎么做呢？

> Mary had a little lamb,  
>   Its fleece was white as snow.   
> And everywhere that Mary went,  
>   The lamb was sure to go.  
> He followed her to school one day,  
>   That was against the rule.  
> It made the children laugh and play  
>   To see a lamb at school.  


---

### **1. 预处理文本**
- 将文本转换为小写（或其他标准化）。
- 去除标点符号、特殊字符。
- 将文本拆分为单词（tokenization）。

---

### **2. 构建词汇表**
- 生成一个单词索引映射（将每个唯一的单词分配一个整数索引）。
- 确保索引唯一，通常使用字典数据结构。

---

### **3. 将文本序列化**
- 使用词汇表将每个单词转换为索引，形成数字化的表示。

---

### **4. 应用Embedding层**
- 使用预训练的嵌入（如GloVe、Word2Vec）或随机初始化的嵌入矩阵将索引转换为向量。

---

### **代码示例：从文本到嵌入向量**

In [2]:
import torch
import torch.nn as nn
import numpy as np
import re
from torch.nn.utils.rnn import pad_sequence

# 1. 输入文本数据
text = """
Mary had a little lamb,
Its fleece was white as snow.
And everywhere that Mary went,
The lamb was sure to go.
He followed her to school one day,
That was against the rule.
It made the children laugh and play
To see a lamb at school.
"""

# 2. 文本预处理
def preprocess_text(text):
    text = text.lower()  # 转小写
    text = re.sub(r'[^a-z\s]', '', text)  # 去除标点符号
    tokens = text.split()  # 拆分成单词
    return tokens

tokens = preprocess_text(text)
print("Tokenized words:", tokens)

Tokenized words: ['mary', 'had', 'a', 'little', 'lamb', 'its', 'fleece', 'was', 'white', 'as', 'snow', 'and', 'everywhere', 'that', 'mary', 'went', 'the', 'lamb', 'was', 'sure', 'to', 'go', 'he', 'followed', 'her', 'to', 'school', 'one', 'day', 'that', 'was', 'against', 'the', 'rule', 'it', 'made', 'the', 'children', 'laugh', 'and', 'play', 'to', 'see', 'a', 'lamb', 'at', 'school']


In [3]:
# 3. 创建词汇表（手动构建词汇索引）
word_to_index = {word: idx + 1 for idx, word in enumerate(sorted(set(tokens)))}
word_to_index['<PAD>'] = 0  # 添加填充标记
print("\nWord Index:", word_to_index)


Word Index: {'a': 1, 'against': 2, 'and': 3, 'as': 4, 'at': 5, 'children': 6, 'day': 7, 'everywhere': 8, 'fleece': 9, 'followed': 10, 'go': 11, 'had': 12, 'he': 13, 'her': 14, 'it': 15, 'its': 16, 'lamb': 17, 'laugh': 18, 'little': 19, 'made': 20, 'mary': 21, 'one': 22, 'play': 23, 'rule': 24, 'school': 25, 'see': 26, 'snow': 27, 'sure': 28, 'that': 29, 'the': 30, 'to': 31, 'was': 32, 'went': 33, 'white': 34, '<PAD>': 0}


In [4]:
# 4. 将文本转换为数字序列
def text_to_sequence(tokens, word_to_index):
    return [word_to_index[word] for word in tokens]

sequences = text_to_sequence(tokens, word_to_index)
print("\nText to sequence:", sequences)


Text to sequence: [21, 12, 1, 19, 17, 16, 9, 32, 34, 4, 27, 3, 8, 29, 21, 33, 30, 17, 32, 28, 31, 11, 13, 10, 14, 31, 25, 22, 7, 29, 32, 2, 30, 24, 15, 20, 30, 6, 18, 3, 23, 31, 26, 1, 17, 5, 25]


In [5]:
# 5. 对序列进行填充
max_len = len(sequences)
sequence_tensor = torch.tensor(sequences, dtype=torch.long)
padded_sequence = torch.cat([sequence_tensor, torch.zeros(max_len - len(sequence_tensor), dtype=torch.long)])

print("\nPadded Sequences:", padded_sequence)


Padded Sequences: tensor([21, 12,  1, 19, 17, 16,  9, 32, 34,  4, 27,  3,  8, 29, 21, 33, 30, 17,
        32, 28, 31, 11, 13, 10, 14, 31, 25, 22,  7, 29, 32,  2, 30, 24, 15, 20,
        30,  6, 18,  3, 23, 31, 26,  1, 17,  5, 25])


In [6]:
# 6. 创建嵌入层（词汇表大小 + 嵌入维度）
vocab_size = len(word_to_index)  # 词汇表大小
embedding_dim = 50  # 嵌入维度，例如50维向量

embedding_layer = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim)

# 7. 获取嵌入结果
embedded_result = embedding_layer(padded_sequence)

print("\nEmbedding shape:", embedded_result.shape)
print("\nEmbedding output for first word:", embedded_result[0].detach().numpy())  # 示例首个单词的嵌入向量


Embedding shape: torch.Size([47, 50])

Embedding output for first word: [ 0.26433146 -0.19831406 -0.47902045 -0.45806032 -0.27450997 -0.33025104
 -1.7811464   0.1441443   0.80690134  1.2267246   0.40146875  0.3023453
 -0.1322729  -0.8056068  -0.32924327  1.3471342   0.7433598   0.26333496
  0.04966741 -0.4080802  -0.11481339 -0.56656396 -1.1198696  -0.0962145
  0.987879    0.18058605  0.22027072 -1.3456892   0.42864525  0.3974306
  0.83492815  0.74255425 -2.505622   -0.32177886  0.5958665   1.602083
  0.5208352  -0.06902823  0.11588498 -0.43345642  0.95750386  0.49879646
 -0.16514462 -1.2297944  -0.12269031  0.7225716   1.0030996  -0.35023418
 -0.05920502  0.20383798]



---

### **解释代码：**
1. **预处理文本**
   - 将文本转换为小写，并去除标点符号。
   - 使用 `split()` 拆分单词。
  
2. **构建词汇表**
   - 使用 `Tokenizer` 将单词映射为索引，例如 `{'mary': 1, 'had': 2, ...}`。

3. **文本序列化**
   - 将文本替换为对应的索引值，例如 `[[1, 2, 3, 4, 5]]`。

4. **填充序列**
   - 确保所有序列长度一致，以便于处理。

5. **嵌入层**
   - 使用 `Embedding(input_dim, output_dim)` 进行随机初始化（或加载预训练权重）。

6. **输出嵌入**
   - 通过 `model.predict` 提取嵌入向量，每个单词映射为固定大小的向量（例如50维）。


### **使用预训练嵌入（如GloVe）**
如果希望使用预训练嵌入而不是随机初始化，可以加载 GloVe 词向量，如下：

```python
# 加载 GloVe 预训练嵌入
embedding_index = {}
with open('glove.6B.50d.txt', 'r', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embedding_index[word] = coefs

# 创建嵌入矩阵
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in word_index.items():
    vector = embedding_index.get(word)
    if vector is not None:
        embedding_matrix[i] = vector

# 使用预训练嵌入层
embedding_layer = Embedding(vocab_size, embedding_dim, weights=[embedding_matrix], input_length=max_len, trainable=False)
```

在这里，`trainable=False` 表示不对嵌入层进行微调，而是直接使用预训练的词向量。

---

### **输出示例：**
```plaintext
Tokenized words: ['mary', 'had', 'a', 'little', 'lamb', 'its', 'fleece', ...]

Word Index: {'mary': 1, 'had': 2, 'a': 3, 'little': 4, 'lamb': 5, ...}

Text to sequence: [[1, 2, 3, 4, 5, 6, 7, 8, ...]]

Padded Sequences:
[[ 1  2  3  4  5  6  7  8  ...]]

Embedding shape: (1, 40, 50)

Embedding output for first word: [-0.23  0.15  0.30  0.12 ...]
```

---

### **总结：**
1. **文本预处理** -> 将文本标准化、分词。
2. **构建词汇表** -> 词到索引的映射。
3. **序列化** -> 将文本转换为数值序列。
4. **应用嵌入层** -> 通过嵌入矩阵获取向量。
5. **选择训练策略**：
   - 训练自己的嵌入。
   - 使用预训练的词向量。

---

希望这个解释和代码示例能够帮助你理解和实现文本到词嵌入的过程！如果有任何疑问，欢迎继续提问。